### 1. 데이터 로드

In [125]:
from datasets import load_dataset
import pandas as pd

news_dataset = load_dataset('nlpHakdang/aihub-news30k',  data_files={"train":"news_train_1_1.csv", "valid":"news_valid_1_1.csv"}, use_auth_token='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')

Using custom data configuration nlpHakdang___aihub-news30k-1ea472361ab2489f
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___aihub-news30k-1ea472361ab2489f/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/2 [00:00<?, ?it/s]

In [126]:
dart_dataset = load_dataset('nlpHakdang/beneficiary',  data_files="dart_v3_3.csv", use_auth_token='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')

Using custom data configuration nlpHakdang___beneficiary-9ee45f4b52a082be
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___beneficiary-9ee45f4b52a082be/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

## 2. 전처리

In [127]:
news_dataset['train']

Dataset({
    features: ['id', 'category', 'media_sub_type', 'media_name', 'char_count', 'publish_date', 'title', 'original', 'article', 'extractive', 'abstract'],
    num_rows: 271093
})

In [128]:
dart_dataset['train']

Dataset({
    features: ['Unnamed: 0', 'index', '주식 코드', 'report_idx', '기업 코드', '기업 이름', '수정 일자', '회사의 개요', '사업의 개요'],
    num_rows: 2313
})

In [129]:
dart_dict = {}
error =0
for dart in dart_dataset['train']:
    try:
        dart_dict[dart['사업의 개요'] + " " + dart['주요 제품 및 서비스']] = dart['기업 이름']
    except:
        error+=1
error        

2313

In [130]:
import pandas as pd

length =500
result = []
cnt = 0
for e in news_dataset['train']:
    if e['category'] in ['경제','기업']:
        for i in range(0, len(e),                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     ):
            result.append(e['article'][i:i+length])
            break
        cnt += 1
    if cnt==1000:
        break


df = pd.DataFrame(result)#.sort_values(by=[0])
df.to_csv('./article_train.csv', index=False)
df

,0
0,중국 투자의향기업 직접 방문 투자협약 체결 협의이성훈 여수광양항만공사 사장 방희석 ...
1,이성훈 포스코 광양제철소 소장 김학동 가 자동차 강판 생산공장인 제3냉연공장을 최신...
2,이성훈 여수광양항만공사 사장 방희석 이하 공사 는 2018년도 승진 및 전보 인사를...
3,지역 특화작목 틈새소득작목 발굴 미래농업 준비이성훈 광양시는 경쟁력을 갖춘 풍요로운...
4,2만2500여명 참여 생명나눔 확산이성훈 포스코 광양제철소 소장 김학동 가 사랑나눔...
...,...
995,GIST 이광희 교수 연구팀 초박막금속 투명전극 이용실생활서 접할 것으로 기대 국제...
996,5년간 다문화 60가족 총 200여명 기회 제공광주은행은 지난 10일 본점 3층에서...
997,2개월 연속 고용률 통계 작성 후 최고치 기록취업자 전년 동월比 광주 6천 전남 1...
998,김장철 수요 증가와 저장비용 상승 등으로 깐마늘과 양파 등 양념류 가격이 오름세를 ...


In [131]:
import pandas as pd

length =500
result = []
cnt = 0
for e in news_dataset['train']:
    if e['category'] in ['경제','기업']:
        for i in range(0, len(e), length):
            result.append(e['article'])
            break
        cnt += 1
    if cnt==1000:
        break

#for e in news_dataset['train']['article'][:1000]:

df2 = pd.DataFrame(result)

## 2. NER

In [132]:
from utils import init_logger, load_tokenizer, get_labels
from predict import *

### 2.1 Evaluation

In [133]:
def predict(pred_config):
    # load model and args
    args = get_args(pred_config)
    device = get_device(pred_config)
    model = load_model(pred_config, args, device)
    label_lst = get_labels(args)
    logger.info(args)

    # Convert input file to TensorDataset
    pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index
    tokenizer = load_tokenizer(args)
    lines = read_input_file(pred_config)
    dataset = convert_input_file_to_tensor_dataset(lines, pred_config, args, tokenizer, pad_token_label_id)

    # Predict
    sampler = SequentialSampler(dataset)
    data_loader = DataLoader(dataset, sampler=sampler, batch_size=pred_config.batch_size)

    all_slot_label_mask = None
    preds = None

    for batch in tqdm(data_loader, desc="Predicting"):
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {"input_ids": batch[0],
                      "attention_mask": batch[1],
                      "labels": None}
            if args.model_type != "distilkobert":
                inputs["token_type_ids"] = batch[2]
            outputs = model(**inputs)
            logits = outputs[0]

            if preds is None:
                preds = logits.detach().cpu().numpy()
                all_slot_label_mask = batch[3].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                all_slot_label_mask = np.append(all_slot_label_mask, batch[3].detach().cpu().numpy(), axis=0)

    preds = np.argmax(preds, axis=2)
    slot_label_map = {i: label for i, label in enumerate(label_lst)}
    print(slot_label_map)
    preds_list = [[] for _ in range(preds.shape[0])]

    for i in range(preds.shape[0]):
        for j in range(preds.shape[1]):
            if all_slot_label_mask[i, j] != pad_token_label_id:
                preds_list[i].append(slot_label_map[preds[i][j]])
    
    # Write to output file
    with open(pred_config.output_file, "w", encoding="utf-8") as f:
        for words, preds in zip(lines, preds_list):
            #line = ""
            line = []
            for word, pred in zip(words, preds):
                if pred == 'O' or pred =='UNK':
                    #line = line + word + " "
                    pass
                else:
                    #line = line + "[{}:{}] ".format(word, pred)
                    if not "기자" in word:
                        #line.append([word, pred])
                        line.append(word)

            f.write("{}\n".format(line))

    logger.info("Prediction Done!")

In [134]:
parser = argparse.ArgumentParser()

parser.add_argument("--input_file", default="article_train.csv", type=str, help="Input file for prediction")
parser.add_argument("--output_file", default="sample_pred_out.txt", type=str, help="Output file for prediction")
parser.add_argument("--model_dir", default="./model", type=str, help="Path to save, load model")

parser.add_argument("--batch_size", default=32, type=int, help="Batch size for prediction")
parser.add_argument("--no_cuda", action="store_false", default=False, help="Avoid using CUDA when available")

pred_config = parser.parse_args(args=[])

In [135]:
predict(pred_config)

<class 'transformers.models.auto.tokenization_auto.AutoTokenizer'> klue/roberta-large


Predicting: 100%|██████████| 32/32 [00:11<00:00,  2.71it/s]


{0: 'UNK', 1: 'O', 2: 'FLD-B', 3: 'FLD-I', 4: 'AFW-B', 5: 'AFW-I', 6: 'ORG-B', 7: 'ORG-I', 8: 'CVL-B', 9: 'CVL-I', 10: 'EVT-B', 11: 'EVT-I', 12: 'TRM-B', 13: 'TRM-I'}


### 2.2. 예측결과 불러오기

In [136]:
ner_list = []
with open("./sample_pred_out.txt", "r") as f:
    for line in f.readlines():
        ner_list.append(line)

In [137]:
ner_list[0:100]

['[]\n',
 "['여수광양항만공사', '사장', 'CEO가', '중남부지역에서', '사장이', 'MOU', '사장은', '푸드존', '운남허메이격치', '주', '콜드체인', '냉동냉장창고', '바이란식품', '주']\n",
 "['포스코', '광양제철소', '소장', '자동차', '제3냉연공장을', '자동차', '3냉연공장', '자동차', '자동차', '자동차']\n",
 "['여수광양항만공사', '사장', '공사', '항만운영팀', '글로벌마케팅팀', '물류단지팀', '정부', '팀장', '항만운영팀장', '부장', '항만개발팀장', '재무회계팀', '물류단지팀', '차장', '가치경영팀', '인재육성팀', '기획조정실', '항만시설팀', '과장', '재무회계팀', '항만개발팀', '주임', '경영지원팀', '항만운영팀']\n",
 "['생강', '취나물', '수박', '아열대소득과수', '애플망고']\n",
 "['포스코', '광양제철소', '소장', '헌혈행사는', '광양제철소와', '대한적십자사', '광주전남혈액원이', '광양제철소에서', '임직원들과', '시민들의', '메일', '직원들의', '직원들에게', '임직원들이']\n",
 "['인공지능', '빅데이터', '스마트팩토리', '포스코', '광양제철소', '소장', '인공지능', '빅데이터', '기술을', '스마트팩토리', '임직원', '포스코의', '사물인터넷', '빅데이터', '인공지능', '직원이', '임직원들은', '광양제철소', '스마트팩토리']\n",
 "['동광양농협', '조합장', '하나로마트가', '대회의실에서', '정기총회를', '하나로마트를', '조합장은', '임직원이']\n",
 "['광양상공회의소', '회장을', '임원단이', '광양시청을', '시장과', '시장은', '회장의', '상의회장은']\n",
 "['시민들', '부영그룹의', '회장이', '목성뜰', '광양읍민들의', '회장은', '특정범죄가중처벌법상', '공정거래법', '임대주택법', '임대주택법', '회

## 3. BM25 문서검색

In [138]:
dart_dataset['train']#['기업 이름']

Dataset({
    features: ['Unnamed: 0', 'index', '주식 코드', 'report_idx', '기업 코드', '기업 이름', '수정 일자', '회사의 개요', '사업의 개요'],
    num_rows: 2313
})

In [139]:
contents_list = ['회사의 개요',  '사업의 개요']#, '주요 제품 및 서비스','주요계약 및 연구개발활동']

dart_dict= {
    "corp_name" :[],
    "corp_text":[]
}

for corp in dart_dataset['train']:
    dart_text = ""
    dart_dict['corp_name'].append(corp['기업 이름'])
    for content in contents_list:
        dart_text += corp[content]
    dart_dict['corp_text'].append(dart_text)


news_list = []
for news in tqdm(news_dataset['train']):
    if news['category'] == '경제' or news['category'] =='기업':
        news_list.append(news['article'])

split_news=[]
for article in news_list:
    split_news.append(article.split("."))


In [140]:
import re
def preprocessing(s): 
        hangul = re.compile('[^ ㄱ-ㅣ가-힣+]')
        result = hangul.sub('', s)
        return result

In [141]:
pre_dart = {}
for idx in range(len(dart_dict['corp_name'])):
    pre_dart[dart_dict['corp_name'][idx]] = preprocessing(dart_dict['corp_text'][idx])
    
pre_name = {}
for idx in range(len(dart_dict['corp_name'])):
    pre_name[preprocessing(dart_dict['corp_text'][idx])] = dart_dict['corp_name'][idx]


In [142]:
from konlpy.tag import Mecab

tokenizer = Mecab()
tot = []
for txt in tqdm(list(pre_dart.values())):
    tot.append(tokenizer.nouns(txt))

100%|██████████| 2313/2313 [00:16<00:00, 139.56it/s]


In [143]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tot)

In [144]:
ner_list[3]

"['여수광양항만공사', '사장', '공사', '항만운영팀', '글로벌마케팅팀', '물류단지팀', '정부', '팀장', '항만운영팀장', '부장', '항만개발팀장', '재무회계팀', '물류단지팀', '차장', '가치경영팀', '인재육성팀', '기획조정실', '항만시설팀', '과장', '재무회계팀', '항만개발팀', '주임', '경영지원팀', '항만운영팀']\n"

### 3.1. NER with BM25

In [145]:
def print_corp(newsindex, dart_dict, bm25, pre_dart, pre_name):
    target_idx=newsindex
    target = df.iloc[target_idx]
    inside_corp = []

    stopwords=['모두','지난','있다','회의','강조','성과','오는','모집','과정','계약',"체결","건립",'현대식','사업',
            '주관사로','통해','일까지','이벤트','지원','지역','기자','실험','한다','진행','설명회','예정이다',
            '이번','방문','올해','제공','찾아가','재림','당시','한국','프로젝트','있는','추진','다양한','적극',
            '위해','나갈','협력','협약','상당의','물품','명으로','지난해','증가했다','의한','영향','보면','통계',
            '순위','명당','관련','업체','부실','관심','명칭','합작법인','일자리','시흥배곧','서울대','했다',
            '한다고','위원장은','기반','혁신','인재','마련','정책','권고안','함께','시장','성장','대비','전월',
            '전년','동월','물가','하락','소비자','상승','운영','증가','억원','억만원','할인','사용','것으']

    tokenized_query = tokenizer.nouns(ner_list[newsindex])
    #print('tokenized_query:',tokenized_query)

    # for key in target.split(" "):
    #     if key in dart_dict['corp_name']:
    #         inside_corp.append(key)

    # for key in tokenized_query:
    #     if key in dart_dict['corp_name']:
    #         inside_corp.append(key)
    
    inside_corp = list(set(inside_corp))
    
    doc_scores = bm25.get_scores(tokenized_query)
    print(doc_scores)
    print(max(doc_scores))
    if max(doc_scores) <3:
        #마땅한 회사가 없음
        return False
    else:
        list_dart_n = bm25.get_top_n(tokenized_query, list(pre_dart.values()), n=5)
        corps = [pre_name[dart] for dart in list_dart_n]
    """
    tokenized_query : keywords
    inside_corp : 언급된 회사
    " ".join(target) : 뉴스텍스트
    corps : topk의 회사이름
    list_dart_n : topk dart
    """
    return tokenized_query, inside_corp, " ".join(target),corps#, list_dart_n


### 3.2. krwordrank with BM25

In [146]:
from krwordrank.word import summarize_with_keywords

def print_corp2(newsindex, dart_dict, bm25, pre_dart, pre_name):
    target_idx=newsindex
    target = df2.iloc[target_idx]
    inside_corp = []

    stopwords=['모두','지난','있다','회의','강조','성과','오는','모집','과정','계약',"체결","건립",'현대식','사업',
            '주관사로','통해','일까지','이벤트','지원','지역','기자','실험','한다','진행','설명회','예정이다',
            '이번','방문','올해','제공','찾아가','재림','당시','한국','프로젝트','있는','추진','다양한','적극',
            '위해','나갈','협력','협약','상당의','물품','명으로','지난해','증가했다','의한','영향','보면','통계',
            '순위','명당','관련','업체','부실','관심','명칭','합작법인','일자리','시흥배곧','서울대','했다',
            '한다고','위원장은','기반','혁신','인재','마련','정책','권고안','함께','시장','성장','대비','전월',
            '전년','동월','물가','하락','소비자','상승','운영','증가','억원','억만원','할인','사용','것으']
    print(target)
    keywords = summarize_with_keywords(target, min_count=4, max_length=7,
        beta=0.85, max_iter=10,stopwords=stopwords,verbose=True)
    print(keywords)
    tokenized_query = list(keywords.keys())

    # for key in target.split(" "):
    #     if key in dart_dict['corp_name']:
    #         inside_corp.append(key)

    # for key in tokenized_query:
    #     if key in dart_dict['corp_name']:
    #         inside_corp.append(key)
    
    inside_corp = list(set(inside_corp))
    
    doc_scores = bm25.get_scores(tokenized_query)
    if max(doc_scores) <5:
        #마땅한 회사가 없음
        return False
    else:
        list_dart_n = bm25.get_top_n(tokenized_query, list(pre_dart.values()), n=5)
        corps = [pre_name[dart] for dart in list_dart_n]
    """
    tokenized_query : keywords
    inside_corp : 언급된 회사
    " ".join(target) : 뉴스텍스트
    corps : topk의 회사이름
    list_dart_n : topk dart
    """
    return tokenized_query, inside_corp, " ".join(target),corps, [e[300:1000] for e  in list_dart_n]

### 3.3 Eval with NER & kwordrank with BM25

In [157]:

#tokenized_corpus = [doc.split(" ") for doc in dart_dict.keys()]
#bm25 = BM25Okapi(tokenized_corpus)
news_id = 9

print_corp(news_id, dart_dict, bm25, pre_dart, pre_name)

[ 6.59009457  3.96333764  6.38593438 ...  7.39197232  3.51869902
 11.06638796]
35.26247017202604


(['시민',
  '부영',
  '그룹',
  '회장',
  '목성',
  '뜰',
  '광양',
  '읍민',
  '회장',
  '특정',
  '범죄',
  '중처',
  '벌',
  '법상',
  '공정',
  '거래법',
  '임대',
  '주택',
  '법',
  '임대',
  '주택',
  '법',
  '회장',
  '회장'],
 [],
 '고용창출 경제 활성화 기대이성훈 지역 업체인 이에스바이오 농업회사법인주식회사 이하 ㈜이에스바이오 대표 강기연 와 광양시는 지난 19일 시민접견실에서 투자협약을 체결했다.이번 투자협약은 광양시가 올해 처음으로 체결한 협약이다.협약식에는 정현복 시장과 강기연 ㈜이에스바이오 대표를 비롯한 관계자 10여명이 참석했다.이번에 신규 투자를 결정한 ㈜이에스바이오는 상토 및 유기질 비료 생산 전문 업체로 분말형상토 압착매트형상토 양액배지 유기질 비료 등 농작물의 종류에 맞는 제품을 생산하는 기술을 보유하고 있으며 생산품을 일본 중국 등지로 수출을 계획하고 있다.이번 협약으로 ㈜이에스바이오는 광양항 동측배후단지 2만9113㎡의 부지에 41억원을 투자해 내년까지 상토 및 유기질 비료 제조 공장을 설립하게 된다.또 광양항 동측배후단지에 입주해 물류비 절감과 수입 원자재에 대한 원가 절감 등을 통한 수익을 극대화해 나갈 예정이다.시는 이번 투자유치를 통해 관리직 생산직을 비롯한 40여명이 양질의 새로운 ',
 ['대원제약', '일성신약', '한신공영', '우리금융지주', '덕양산업'])

In [154]:
print_corp2(news_id, dart_dict, bm25, pre_dart, pre_name)

0    정기총회 개최 조합원 실익 사업 최선 이성훈 동광양농협 조합장 이명기 하나로마트가 ...
Name: 6, dtype: object
scan vocabs ... 
num vocabs = 25
done = 10
{'조합원': 1.5712066401602494, '최선': 1.1489907352542938}


(['조합원', '최선'],
 [],
 '정기총회 개최 조합원 실익 사업 최선 이성훈 동광양농협 조합장 이명기 하나로마트가 6년 연속 총매출 500억원을 돌파했다.동광양농협은 지난달 26일 3층 대회의실에서 2018년 정기총회를 개최했다.동광양농협은 지난해 다양한 금융서비스 제공과 하나로마트를 통한 우수농산물 판매로 순이익 21억원 총자산 4713억원 이상을 달성했다고 밝혔다.또한 상호금융예수금 4174억원과 상호금융대출금 3327억원을 달성했다.하나로마트 모든 사무소 총 매출액은 517억원으로 6년 연속 500억원 이상을 달성했다.이명기 조합장은 지난 1년은 조합원 여러분과 약속했던 일들을 기필코 달성하기 위해 전 임직원이 일심단결했다 면서 건전하고 투명한 경영을 통한 자립경영 기반강화를 탄탄히 다지는 한 해였다 고 밝혔다.이 조합장은 임직원은 혼연일체가 되어 무술년 한 해 새롭게 농협을 창립한다는 정신으로 조합원과 고객과의 협력을 통해 최고의 목표인 건전결산의 결실을 이루는 한 해가 되도록 최선을 다하겠다 고 다짐했다.이어 조합원 삶의 질을 높이고 작은 소리에도 귀 기울여 업무수행에 최선을 다할 것 이라며 우리 조합을 믿고 찾는 고객을 최고로 모시며 민원 없는 업무수행과 조합원 실익사업에 최선을 다하겠다 고 강조했다.동광양농협은 지난해 우수조합원 해외역사문화 탐방 조합원 건강검진 추곡 자체수매차액 지원 영농자재 구입용 교환권 공급 조합원 자녀장학금 지원 등 다양한 환원사업을 실시 농업인 조합원의 삶의 질 향상과 농가실익 증대를 위해 노력했다.',
 ['미래에셋벤처투자', '컴퍼니케이', '나우IB', '삼호개발', '디피씨'],
 [' 사업을 영위할 목적으로 년 월 일 설립되었습니다  년 월경영참여형사모집합투자기구  의 업무집행사원 등록을 완료함으로써 사업의 범위를 확대해 가고 있습니다  당사는 별도의 존속기간을 정하고 있지 않습니다  바  본사의 소 전화번호 홈페이지 소 소   경기도 성남시 분당구 판교역로번길  전화번호    홈페이지   사  회사사업 